In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import sklearn
import sklearn.ensemble
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
import csv

In [ ]:
path = '../datas/nlp_results/'
voting_df = pd.read_csv(path+'voting_with_topics.csv')
print('Entries in the DataFrame',voting_df.shape)

#Dropping the useless column
voting_df = voting_df.drop('Unnamed: 0',1)

#Putting numerical values into the columns that should have numerical values
#print(voting_df.columns.values)

num_cols = ['Decision', ' armée', ' asile / immigration', ' assurances', ' budget', ' dunno', ' entreprise/ finance',
           ' environnement', ' famille / enfants', ' imposition', ' politique internationale', ' retraite  ']
voting_df[num_cols] = voting_df[num_cols].apply(pd.to_numeric)

#Inserting the full name at the second position
voting_df.insert(2,'Name', voting_df['FirstName'] + ' ' + voting_df['LastName'])

voting_df = voting_df.drop_duplicates(['Name'], keep = 'last')
voting_df = voting_df.set_index(['Name'])
voting_df.head(3)

In [ ]:
profileMatrixFile = 'profileMatrix.csv'
profileMatrix = pd.read_csv(profileMatrixFile, index_col = 0)
profileArray = profileMatrix.values
print(profileArray.shape)
profileMatrix.head()

In [ ]:
distanceMatrixFile = 'distanceMatrix.csv'
distances = pd.read_csv(distanceMatrixFile, index_col = 0)
distances = distances.replace(-0.001, 0)
distancesArray = distances.values
print(distancesArray.shape)
distances.head()

In [ ]:
distances.values

In [ ]:
from scipy.sparse import csr_matrix
distanceSparse = csr_matrix(distances.values)
distanceSparse

In [ ]:
 kmeans = KMeans(n_clusters=8, init = 'k-means++', precompute_distances = True, random_state=0).fit(profileMatrix)

In [ ]:
predictions = kmeans.labels_

print("Silhouette score : {s}".format(s=silhouette_score(profileMatrix, kmeans.labels_)))

In [ ]:
predictions

In [ ]:
#spectClust = sklearn.cluster.SpectralClustering(n_clusters = 8, affinity = 'precomputed')

nbClust = 3
clusterDist = sklearn.cluster.spectral_clustering(affinity = distanceSparse, n_clusters = nbClust)
#.fit(profileArray)
#spectClust.affinity_matrix_ = distancesArray
#spectClust.fit(distancesArray)
clusterDist

In [ ]:
distances.index[clusterDist == 0]

For each cluster, compute the ratio for each partite

In [ ]:
ratio_df = pd.DataFrame(index = voting_df.ParlGroupName.unique())
ratio_df['ratio'] = 0
np.array(ratio_df.index)

In [ ]:
people = distances.index[clusterDist == 2]
group = 'Groupe conservateur-catholique'
[voting_df.loc[people].ParlGroupName == group]

In [ ]:
def ratioPartite(cluster, clusterDist):
    # Compute the partites distribution for all people within this cluster
    people = distances.index[clusterDist == cluster]
    size = len(people)
    ratio_df = pd.DataFrame(index = voting_df.ParlGroupName.unique())
    ratio_df['ratio'] = 1.0
    for group in np.array(ratio_df.index):
        print(group)
        peopleGroup = [p for p in people[voting_df.loc[people].ParlGroupName == group]]
        print(len(peopleGroup) / float(size))
        ratio_df.set_value(group, 'ratio', len(peopleGroup) / float(size))
    return ratio_df
    

In [ ]:
ratio_df = ratioPartite(0, clusterDist)
ratio_df.values

In [ ]:
ratio_df = pd.DataFrame(index = voting_df.ParlGroupName.unique(), columns = range(nbClust))
ratio_df[0] = range(8)
ratio_df

In [ ]:
ratio_df = pd.DataFrame(index = voting_df.ParlGroupName.unique(), columns = range(nbClust))
for cluster in range(nbClust):
    ratio = ratioPartite(cluster, clusterDist)
    ratio_df[cluster] = ratio.values
    
ratio_df